### Geometry

First, we'll load in a GeoJSON file describing the outline of Pine Island Ice Shelf.
This outline was hand-digitized from the various input data sets we'll use in a GIS.

In [ ]:
import geojson
outline_filename = 'pine-island-shelf.geojson'
with open(outline_filename, 'r') as outline_file:
    outline = geojson.load(outline_file)

Next we'll use a few utility functions from icepack to turn this outline into the input format for the mesh generator [gmsh](https://www.gmsh.info).
We can then generate an unstructured triangular mesh of the domain and load in that mesh.

In [ ]:
import icepack
geometry = icepack.meshing.collection_to_geo(outline)
with open('pine-island-shelf.geo', 'w') as geo_file:
    geo_file.write(geometry.get_code())

In [ ]:
!gmsh -2 -format msh2 -v 2 -o pine-island-shelf.msh pine-island-shelf.geo

In [ ]:
import firedrake
mesh = firedrake.Mesh('pine-island-shelf.msh')

The colors correspond to the numeric IDs of each boundary segment (note the legend in the corner).
We need a way of identifying different boundary conditions in the ice shelf flow model in order to determine where ice is flowing in from and where the terminus is.

In [ ]:
import icepack.plot
fig, axes = icepack.plot.subplots()
icepack.plot.triplot(mesh, axes=axes)
axes.legend();

### Input data

Next we have to load in some observational data for the ice shelf draft.
The demos for icepack include a module to fetch the most common observational data sets.
We'll use BedMachine Antarctica because they've already done the hard work of things like firn and geoid corrections to ice shelf thickness.
This routine will download the BedMachine dataset from NSIDC.
If this is your first time running this notebook, you'll be prompted for your EarthData username and password.

In [ ]:
bedmachine_filename = icepack.datasets.fetch_bedmachine_antarctica()

To get the ice shelf draft, we'll first get the surface elevation and thickness of the ice shelf.

In [ ]:
import rasterio
surface_grid = rasterio.open('netcdf:' + bedmachine_filename + ':surface', 'r')
thickness_grid = rasterio.open('netcdf:' + bedmachine_filename + ':thickness', 'r')

We'll represent the ice shelf draft using continuous, piecewise quadratic basis functions.

In [ ]:
Z = firedrake.FunctionSpace(mesh, family='CG', degree=2)

Next we'll interpolate the gridded data to our finite element space.

In [ ]:
surface = icepack.interpolate(surface_grid, Z)
thickness = icepack.interpolate(thickness_grid, Z)

Finally we can get the ice shelf draft as the difference of the surface and the thickness.

In [ ]:
z_obs = firedrake.interpolate(surface - thickness, Z)

In [ ]:
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tricontourf(z_obs, 40, axes=axes)
fig.colorbar(contours);

In the interest of keeping our numerical solver from losing its bloody mind, we'll smooth over the ice shelf.
The spin-up of the plume will proceed in two stages.
First, we'll use a very smooth ice shelf draft.
Most of the interesting features, like sub-ice shelf channels, will have been diffused out.
But this will make it easier to evolve the plume towards a sane steady state than if we had used the real data as-is.
In the next phase, we'll slowly morph the ice shelf draft towards a value with a much lower smoothing length in order to recapture more of the features of the real data.

In [ ]:
from firedrake import inner, grad, dx, ds, Constant
z = firedrake.Function(Z)
α = Constant(5e3)

parameters = {
    'solver_parameters': {
        'ksp_type': 'preonly',
        'pc_type': 'lu',
        'pc_factor_mat_solver_type': 'mumps'
    }
}

J = 0.5 * ((z - z_obs)**2 + α**2 * inner(grad(z), grad(z))) * dx
F = firedrake.derivative(J, z)
firedrake.solve(F == 0, z, **parameters)

This is enough smoothing to keep the plume solver from exploding, but not so much as to wipe out features like sub-ice shelf channels.

In [ ]:
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tricontourf(z, 40, axes=axes)
fig.colorbar(contours);

Some amount of smoothing is also usually necessary for the ice flow model too.
Newer remote sensing platforms like ICESat-2 are sophisticated enough to resolve individual crevasses, introducing sharp breaks in the ice thickness.
The gradient of the ice thickness is one of the sources of the ice flow model, so these features, which are of too small a scale to really influence the flow by themselves, end up breaking the numerics.

### Inflow data

Christine Dow at the University of Waterloo has graciously provided us with some output from a hydrology model of the water flux out of the grounding line and into the plume.
The total water influx from the model across all channels is 35.64 m${}^3$/s.
The maximum influx of any one channel is 30.46 m${}^3$/s with a velocity of 0.86 m/s, so if you figure that the channel is shaped liked a half-circle, this shakes out to a radius of 4.75m.
The largest channel is centered at the point (x, y) = -1589${}^3$km, -255.5${}^3$km in this coordinate system, which is right near the deepest point of the grounding line.
There are two smaller channels about 6km away to either side.

We won't try to simulate these basal water channels as the point-like sources that they are because this is too hard.
Instead, we'll use an inflow plume velocity and thickness that reproduce the net flux from the hydrology model, but with a profile that's smoothed out over a few grid cells.

In [ ]:
max_inflow_point = Constant((-1589e3, -255.5e3))
radius = Constant(10e3)
grounding_line_flux = Constant(35.64)

In [ ]:
def sech(z):
    return 2 / (firedrake.exp(z) + firedrake.exp(-z))

n = firedrake.FacetNormal(mesh)
x = firedrake.SpatialCoordinate(mesh)
δx = x - max_inflow_point
r = firedrake.sqrt(inner(δx, δx))
shape = -sech(r / radius) * n
total = firedrake.assemble(inner(shape, -n) * ds((1,)))
q_in = Constant(grounding_line_flux / total) * shape

In [ ]:
max_inflow_thickness = Constant(4.0)
h_in = max_inflow_thickness * sech(r / radius)

### Initial state

Next, we need to come up with some vaguely sane initial state of the plume.
The model can quickly explode if we initialize it with a weird initial state that has large transients, so we need to find something vaguely reasonable.
Roughly speaking, the fluid velocity should align with the gradient of the ice shelf bottom.

In [ ]:
W = firedrake.VectorFunctionSpace(mesh, family='CG', degree=2)
grad_z = firedrake.project(grad(z), W)

To make the initial velocity field, we'll solve an optimization problem.
The objective functional will favor a velocity field that is:

1. smoothly varying
2. non-divergent
3. aligns with the ice shelf draft gradient whenever it points into the domain

Putting all of these criteria together, we get the objective 

$$J = \frac{\alpha^2}{2}\int_\Omega\left\{|\nabla u|^2 + (\nabla\cdot u)^2\right\}dx + \frac{\alpha}{2}\int_\Gamma |u - v|^2ds$$

where $\Gamma$ is everything except the ice shelf terminus, $v$ is the inflow vector

$$v = \max\{0, \nu\cdot\nabla z_b\}\nu,$$

and $\nu$ the unit inward-pointing normal to the domain.

In [ ]:
from firedrake import div
v = firedrake.Function(W)

n = -firedrake.FacetNormal(mesh)
v_n = firedrake.max_value(inner(grad_z, n), 0) * n

terminus_ids = (2, 3, 4)
J = (
    0.5 * α**2 * (inner(grad(v), grad(v)) + div(v)**2) * dx +
    0.5 * α * inner(v - v_n, v - v_n) * ds(terminus_ids)
)
F = firedrake.derivative(J, v)
firedrake.solve(F == 0, v, **parameters)

As a sanity check we can make a stream plot of the initial value of the plume velocity.
The streamlines get thicker towards the end, so this vector field actually is going from the inflow boundary to the terminus as we had hoped.

In [ ]:
fig, axes = icepack.plot.subplots()
firedrake.triplot(
    mesh, axes=axes,
    boundary_kw={'color': 'k'},
    interior_kw={'linewidth': 0.05}
)
streamlines = firedrake.streamplot(v, resolution=2e3, seed=1, axes=axes)
fig.colorbar(streamlines);